In [ ]:
import pandas as pd
import pyodbc
dfspecies= pd.read_csv("C:/Users/Maria Sirevaag/Documents/Pythonprojects/Natura2000_end2020_csv/Natura2000_end2020_SPECIES.csv", header= 0, )

print(dfspecies)


### Create a Database in Sql Server
NAMED: Natura2000

## Create Dim_Species

In [ ]:
species_toclean= dfspecies.iloc[:, [2,3,5]]

#species_toclean = species_toclean.dropna()  #delete all rows with nan
species_toclean

In [ ]:
species_toclean=species_toclean.drop_duplicates() #delete all rows with duplicates

species_toclean['SpeciesId'] =range(1, len(species_toclean) + 1)

In [ ]:
species_toclean[['Genus','Specie']] = species_toclean['SPECIESNAME'].str.split(' ',1,expand=True,)
species_toclean = species_toclean.rename(columns = {'SPECIESCODE': 'SpeciesCode'}, inplace = False)
species_toclean.drop('Specie', inplace=True, axis=1)
species_toclean = species_toclean.rename(columns = {'SPGROUP': 'Class'}, inplace = False)
species_toclean = species_toclean.rename(columns = {'SPECIESNAME': 'SpeciesName'}, inplace = False)
species_dim= species_toclean[['SpeciesId','SpeciesName','Genus','SpeciesCode','Class']]
species_dim


In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-9SHPBD0;'
                      'Database=Natura2000;'
                      'Trusted_Connection=yes;'
                     )


cursor = conn.cursor()


In [ ]:
cursor.execute('''
		CREATE TABLE Dim_species (
			SpeciesID int primary key,
			SpeciesName varchar(30),
            Genus varchar(30),
            SpecieCode varchar(5),
            Class varchar(30)
			
			)
               ''')

conn.commit()

In [ ]:
from sqlalchemy import create_engine
import urllib
import pyodbc

quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=DESKTOP-9SHPBD0;DATABASE=Natura2000")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

species_dim.to_sql('Dim_species', schema='dbo', con = engine, chunksize=200, method='multi', index=False, if_exists='replace')

## create a Region dimension

In [ ]:
region_toclean= dfspecies.iloc[:, [0,]]
region_toclean=region_toclean.drop_duplicates()
region_toclean = region_toclean.rename(columns = {'COUNTRY_CODE': 'Code'}, inplace = False)

In [ ]:

region_toclean
#convert first or second to str or int
region_toclean['Code'] = region_toclean['Code'].astype(str)

In [ ]:
dfcountries= pd.read_csv(r"C:/Users/Maria Sirevaag/Documents/Pythonprojects/countries_codes_and_coordinates.csv", header= 0, )
dfcountries= dfcountries.iloc[:, [0,1,4,5]]

dfcountries = dfcountries.rename(columns = {'Alpha-2 code': 'Code'}, inplace = False)
dfcountries = dfcountries.rename(columns = {'Latitude (average)': 'Latitude'}, inplace = False)
dfcountries = dfcountries.rename(columns = {'Longitude (average)': 'Longitude'}, inplace = False)
dfcountries=dfcountries.drop_duplicates() #delete all rows with duplicates

In [ ]:
def trim_all_columns(df):  #    Trim whitespace from ends of each value across all series in dataframe
  
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

# trimming whitespace from data elements
dfcountriest = pd.DataFrame(dfcountries)
dfcountriest = trim_all_columns(dfcountries)
print(dfcountries)


In [ ]:
def trim_all_columns(df):#    Trim whitespace from ends of each value across all series in dataframe
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)


# trimming whitespace from data elements
region_tocleant = pd.DataFrame(region_toclean)
region_tocleant = trim_all_columns(region_toclean)
print(region_tocleant)


In [ ]:
result_df = pd.merge(region_tocleant, dfcountriest, on = "Code",  how = "inner")

In [ ]:
result_df


In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-9SHPBD0;'
                      'Database=Natura2000;'
                      'Trusted_Connection=yes;'
                     )


cursor = conn.cursor()




In [ ]:
cursor.execute('''
		CREATE TABLE Dim_Geography (
			Code nvarchar(5),
            Country nvarchar(30),
            Latitude nvarchar (30),
            Longitude nvarchar(30)
			)
               ''')

conn.commit()


In [ ]:
from sqlalchemy import create_engine
import urllib
import pyodbc

quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=DESKTOP-9SHPBD0;DATABASE=Natura2000")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

result_df.to_sql('Dim_Geography', schema='dbo', con = engine, chunksize=200, method='multi', index=False, if_exists='replace')

## Create a timeDimension Table


In [ ]:
Time=pd.date_range('2006-01-01','2020-12-31', 
              freq='MS').strftime("%Y-%b")

Time_dim= pd.DataFrame(Time)

Time_dim[['Year','month']] = Time_dim[0].str.split('-',expand=True)
Time_dim= Time_dim.rename(columns = {Time_dim.columns[0]: 'TimeId'}, inplace = False)



Time_dim


In [ ]:
 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-9SHPBD0;'
                      'Database=Natura2000;'
                      'Trusted_Connection=yes;'
                     )


cursor = conn.cursor()




In [ ]:
cursor.execute('''
		CREATE TABLE Time_dim (
            TimeId varchar (20),
			Year varchar(10),
            Month nvarchar(5)
			)
               ''')

conn.commit()


In [ ]:
from sqlalchemy import create_engine
import urllib
import pyodbc

quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=DESKTOP-9SHPBD0;DATABASE=Natura2000")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

Time_dim.to_sql('Time_dim', schema='dbo', con = engine, chunksize=200, method='multi', index=False, if_exists='replace')

### CREATE A TABLE SITECODE

In [ ]:
import pyodbc
cnxn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\Users\Maria Sirevaag\Documents\Natura2000\Natura2000_end2020.mdb;')
query = "select COUNTRY_CODE, SITECODE, SITETYPE,SITENAME,DATE_UPDATE,AREAHA, LONGITUDE, LATITUDE FROM NATURA2000SITES"
site_toclean = pd.read_sql(query, cnxn)
cnxn.close()

site_toclean


In [ ]:

site_toclean[['DateUpd','time']] = site_toclean['DATE_UPDATE'].astype(str)    .str.split(' ',expand=True)
site_toclean.drop('DATE_UPDATE', inplace=True, axis=1)
site_toclean.drop('time', inplace=True, axis=1)
site_toclean = site_toclean.rename(columns = {'COUNTRY_CODE': 'CountryCode'}, inplace = False)
site_toclean = site_toclean.rename(columns = {'SITECODE': 'SiteCode'}, inplace = False)
site_toclean = site_toclean.rename(columns = {'SITETYPE': 'SiteType'}, inplace = False)
site_toclean = site_toclean.rename(columns = {'SITENAME': 'SiteName'}, inplace = False)
site_toclean = site_toclean.rename(columns = {'AREAHA': 'AreaHa'}, inplace = False)
site_toclean = site_toclean.rename(columns = {'LONGITUDE': 'Longitude'}, inplace = False)
site_toclean = site_toclean.rename(columns = {'LATITUDE': 'Latitude'}, inplace = False)



In [ ]:

site_dim= site_toclean[['SiteCode','SiteName','SiteType','CountryCode','AreaHa','Longitude','Latitude','DateUpd']]

site_dim

In [ ]:


cursor.execute('''
		CREATE TABLE Dim_site (
			SiteCode varchar(10) primary key,
            SiteName varchar(50),
            SiteType varchar(10),
            CountryCode varchar(10),
            AreaHa varchar(10),
            Longitude Decimal (10,8),
            Latitude Decimal (10,8),
            DateUpd date
			)
               ''')

conn.commit()




In [ ]:
from sqlalchemy import create_engine
import urllib
import pyodbc

quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=DESKTOP-9SHPBD0;DATABASE=Natura2000")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

site_dim.to_sql('Dim_site', schema='dbo', con = engine, chunksize=200, method='multi', index=False, if_exists='replace')